In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
import math
import descartes
from shapely.geometry import Point,Polygon
death_table = pd.read_csv('out.csv', sep=',').transpose().drop('Unnamed: 0')
death_table.index
resident_data_2018_raw = pd.read_excel('ukmidyearestimates20182019ladcodes.xls', 'MYE2-All', skiprows=4)
resident_data_2018_raw=resident_data_2018_raw.rename(columns={'Code':'AreaCode'})
resident_data_2018_raw =resident_data_2018_raw[resident_data_2018_raw['AreaCode'].isin(death_table.index)]
resident_data_2018_raw=resident_data_2018_raw.reset_index()
TotalPop=resident_data_2018_raw['All ages']
resident_data_2018_raw=resident_data_2018_raw.drop(columns=['index',"Name", "Geography1",'All ages'])
# resident_data_2018=resident_data_2018_raw.drop(columns=['AreaCode'])


In [5]:
p[0]

1.87571171e-06

In [16]:
resident_data_2018_raw

,AreaCode,0,1,2,3,4,5,6,7,8,...,81,82,83,84,85,86,87,88,89,90
0,E06000047,4989.0,5252.0,5448.0,5547.0,5785.0,5939.0,5929.0,6223.0,6140.0,...,3312.0,2843.0,2659.0,2341.0,2063.0,1811.0,1741.0,1409.0,1149.0,4138.0
1,E06000005,1134.0,1104.0,1207.0,1216.0,1256.0,1314.0,1369.0,1284.0,1341.0,...,664.0,613.0,594.0,494.0,489.0,407.0,342.0,301.0,259.0,1058.0
2,E06000001,999.0,1011.0,1101.0,1060.0,1126.0,1136.0,1200.0,1219.0,1218.0,...,595.0,491.0,491.0,427.0,405.0,351.0,289.0,267.0,217.0,787.0
3,E06000002,1871.0,1970.0,1882.0,1969.0,1965.0,2025.0,1955.0,1975.0,1911.0,...,714.0,676.0,608.0,531.0,525.0,392.0,421.0,323.0,280.0,886.0
4,E06000057,2874.0,2944.0,3058.0,2945.0,3089.0,3284.0,3368.0,3442.0,3481.0,...,2262.0,2011.0,1953.0,1632.0,1484.0,1353.0,1211.0,1079.0,864.0,3347.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,S12000028,938.0,952.0,1061.0,1107.0,1039.0,1074.0,1087.0,1206.0,1159.0,...,858.0,784.0,736.0,609.0,604.0,507.0,476.0,383.0,311.0,1248.0
367,S12000029,3198.0,3358.0,3343.0,3472.0,3432.0,3533.0,3604.0,3669.0,3466.0,...,1803.0,1742.0,1458.0,1373.0,1254.0,1081.0,983.0,775.0,691.0,2331.0
368,S12000030,815.0,802.0,901.0,856.0,920.0,940.0,992.0,975.0,958.0,...,592.0,498.0,498.0,376.0,373.0,357.0,244.0,212.0,206.0,720.0
369,S12000039,867.0,909.0,963.0,929.0,958.0,1032.0,1053.0,1050.0,1034.0,...,465.0,448.0,387.0,362.0,327.0,304.0,245.0,211.0,193.0,573.0


In [39]:
p=[1.87571171e-06, 3.54961995e+00]
totdeath=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(totdeath)):
    for j in range(0,90):
        totdeath[i]=totdeath[i]+p[0]*np.power(j,p[1])*resident_data_2018_raw.iloc[i][j]
averagedeathRate=totdeath/TotalPop/100

In [58]:
p=[0.65850154,-50.5052005 ]
totasymp=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(resident_data_2018_raw)):
    for j in range(0,90):
        totasymp[i]=totasymp[i]+p[0]*np.power(np.exp(1),j/p[1])*resident_data_2018_raw.iloc[i][j]
averageasymp=totasymp/TotalPop

In [72]:
p=[0.01022282, 0.85039945]
tothh=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(resident_data_2018_raw)):
    for j in range(0,90):
        tothh[i]=tothh[i]+p[0]*np.power(j,p[1])*resident_data_2018_raw.iloc[i][j]
averagehh=tothh/TotalPop

In [56]:
p=[0.02257974, 0.71334969]
totcc=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(resident_data_2018_raw)):
    for j in range(0,90):
        totcc[i]=totcc[i]+p[0]*np.power(j,p[1])*resident_data_2018_raw.iloc[i][j]
averagecc=totcc/TotalPop

In [54]:
p=[5.94788113e-17, 8.51304632e+00]
totmm=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(resident_data_2018_raw)):
    for j in range(0,90):
        totmm[i]=totmm[i]+p[0]*np.power(j,p[1])*resident_data_2018_raw.iloc[i][j]
averagemm=totmm/TotalPop

In [73]:
averagecc*averagemm*averagehh

0      0.009757
1      0.009643
2      0.008998
3      0.006866
4      0.012534
         ...   
366    0.013368
367    0.009357
368    0.009012
369    0.008831
370    0.007108
Name: All ages, Length: 371, dtype: float64

In [83]:
AgeRate= pd.DataFrame((resident_data_2018_raw['AreaCode']))
# AgeRate=AgeRate.transpose()
# AgeRate
# AgeRate=AgeRate.rename(columns={'All ages':'CFR'})
# AgeRate['CFR']=averagedeathRate
AgeRate['Asymp']=averageasymp
AgeRate['HH']=averagehh
AgeRate['CC']=averagecc
AgeRate['MM']=averagemm
AgeRate.to_csv('AgeRate.csv')

In [84]:
AgeRate

,AreaCode,Asymp,HH,CC,MM
0,E06000047,0.319179,0.235214,0.306376,0.135399
1,E06000005,0.321094,0.232861,0.303431,0.136481
2,E06000001,0.325578,0.229826,0.300049,0.130485
3,E06000002,0.343110,0.216310,0.284625,0.111520
4,E06000057,0.303634,0.247607,0.320165,0.158112
...,...,...,...,...,...
366,S12000028,0.301112,0.249603,0.322395,0.166125
367,S12000029,0.317712,0.236154,0.307561,0.128826
368,S12000030,0.322022,0.232162,0.303271,0.127994
369,S12000039,0.320799,0.233904,0.305091,0.123754


In [10]:
totage=np.zeros(len(resident_data_2018_raw))
for i in range(0,len(resident_data_2018_raw)):
    for j in range(0,90):
        totage[i]=totage[i]+p[0]*np.power(j,p[1])*resident_data_2018_raw.iloc[i][j]

In [11]:
averageage=totage/TotalPop
averageage.max()

3.621182524487075